<a href="https://colab.research.google.com/github/mdehghani86/AppliedGenAI/blob/main/M11_Lab1_CrewAI_2_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>CrewAI Lab</title>
  <style>
    body {
      font-family: 'Segoe UI', sans-serif;
      background-color: #f8fbff;
      color: #222;
      padding: 30px;
      line-height: 1.6;
    }

    h1, h2 {
      color: #1e4b8f;
    }

    a {
      color: #1e4b8f;
      text-decoration: none;
    }

    a:hover {
      text-decoration: underline;
    }

    .center {
      text-align: center;
    }

    .title-img {
      width: 600px;
      max-width: 100%;
      margin-bottom: 20px;
    }

    .table-container {
      overflow-x: auto;
      margin-top: 20px;
    }

    table {
      border-collapse: collapse;
      width: 100%;
      background-color: #fff;
      box-shadow: 0 2px 5px rgba(0,0,0,0.05);
    }

    th, td {
      padding: 12px 16px;
      text-align: left;
      border-bottom: 1px solid #e0e0e0;
    }

    th {
      background-color: #e6f0fb;
      color: #1e4b8f;
    }

    ul {
      margin-top: 10px;
    }
  </style>
</head>
<body>

  <div class="center">
    <img src="https://www.dropbox.com/scl/fi/72c3k12bjnhxdebypttum/crewAI_title.png?dl=1" alt="crewAI Title" class="title-img">
  </div>

  <h1>Welcome to the CrewAI Lab</h1>

  <p>
    This lab introduces <strong>CrewAI</strong>, a powerful platform for orchestrating collaborative AI agents, and explores its integration with LangChain and n8n.
    Visit the <a href="https://www.crewai.com/" target="_blank">CrewAI website</a> to learn more about how CrewAI empowers teams to build complex, automated workflows and advanced_


# 🤖 CrewAI 2-Agent Tutorial
## What is CrewAI?

**CrewAI** is a Python framework that creates teams of AI agents working together. Think of it as hiring specialists - each agent has a specific role and expertise.

### 🏗️ Core Components:
1. **🤖 Agents**: AI team members with specific roles
2. **📋 Tasks**: Individual assignments
3. **👥 Crew**: Manages the team workflow
4. **🔧 Tools**: Special abilities (web search, calculations, etc.)

---

In [ ]:
# +++++ 🎨 Pretty Print Utility
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Create styled message boxes for better visual output in Colab

from IPython.display import display, HTML

def pretty_print(text, title="ℹ️ Info", theme="blue"):
    """Displays a styled message box with optional color themes: blue, red, or yellow."""

    themes = {
        "blue": {"color": "#1e4b8f", "background": "#f0f6ff"},
        "red": {"color": "#c62828", "background": "#ffebee"},
        "yellow": {"color": "#b26a00", "background": "#fff8e1"}
    }

    style = themes.get(theme.lower(), themes["blue"])
    formatted_text = text.replace('\n', '<br>')

    display(HTML(f"""
    <div style="border-left: 5px solid {style['color']}; padding: 12px 16px; background-color: {style['background']};
                border-radius: 6px; font-family: 'Segoe UI', sans-serif; line-height: 1.6; margin: 10px 0;">
        <strong style="color: {style['color']}; font-size: 16px;">{title}</strong><br>
        <span style="font-size: 14px; color: #333;">{formatted_text}</span>
    </div>
    """))

print("🎨 Pretty print utility ready!")

In [ ]:
# +++++ 📦 Installation & API Setup
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Install packages and setup secure API key

!pip install crewai crewai-tools -q

try:
    from google.colab import userdata
    import os
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    pretty_print("🔐 API key loaded successfully!", "✅ Setup Complete", "blue")
except:
    pretty_print("⚠️ Please set OPENAI_API_KEY in Colab → More → Secrets", "❌ Missing API Key", "red")

## 🤖 Defining Agents in CrewAI

In CrewAI, each **agent** represents an autonomous expert responsible for a specific task.  
You define agents using three main components:

- **Role 🎯:** Clearly describes the agent's professional role or expertise.
- **Goal 🎯:** Defines the specific objective the agent should achieve.
- **Backstory 📖:** Provides a short narrative about the agent’s expertise or experience, giving context to their behavior.
## Understanding 2-Agent System

We'll create **2 specialized agents**:
- **🔍 Research Agent**: Gathers information using web search
- **📝 Analysis Agent**: Analyzes and summarizes findings

In [ ]:
# +++++ 🤖 Creating 2 Generalized Agents (NO TOOLS)
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Define two generalized agents that can handle any topic

from crewai import Agent

# 🔍 Agent 1: Research Agent (Generalized)
research_agent = Agent(
    role="Research Specialist",
    goal="Gather comprehensive information about any given topic using existing knowledge",
    backstory="""You are an experienced researcher with broad knowledge across many fields.
    You excel at providing detailed, accurate information on various topics from your training data.""",
    verbose=True,
    allow_delegation=False
)

# 📝 Agent 2: Analysis Agent (Generalized)
analysis_agent = Agent(
    role="Content Analyst",
    goal="Analyze and summarize information into clear, concise insights",
    backstory="""You are an expert analyst who can synthesize complex information into
    actionable insights. You excel at creating structured summaries and key takeaways.""",
    verbose=True,
    allow_delegation=False
)

pretty_print("🤖 Two generalized agents created successfully!\n🔍 Research Agent - Gathers information\n📝 Analysis Agent - Synthesizes findings\n\n⚠️ Note: No tools yet - agents use existing knowledge", "✅ Agents Ready", "blue")

## 📋 What is a Task?

In CrewAI, a **Task** is a unit of work assigned to an agent. It describes what the agent should do and what kind of output is expected.

## 🛠️ What’s happening here?

We define two tasks:
- A **Research Task** asking the agent to gather detailed facts about the topic.
- An **Analysis Task** asking another agent to summarize and interpret the findings.

The `TOPIC` variable makes both tasks reusable for any subject by simply changing one line.


In [ ]:
# +++++ 📋 Creating Tasks with Topic Variable
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

from crewai import Task  # Only import Task here

# Define your topic of interest
TOPIC = "Harvard University's impact on education and Nobel Prize connections"

# 🧠 Task 1: Research Task
research_task = Task(
    description=f"""Research the topic: {TOPIC}

    Provide comprehensive information including:
    1. Key facts and background information
    2. Important developments and achievements
    3. Current status and recent updates
    4. Significance and impact

    Use your existing knowledge to provide accurate, detailed information.""",
    expected_output="Comprehensive research findings about the topic",
    agent=research_agent
)

# 🔎 Task 2: Analysis Task
analysis_task = Task(
    description=f"""Analyze the research findings about: {TOPIC}

    Create a structured summary with:
    1. One paragraph overview (3-4 sentences)
    2. 3-4 key bullet points highlighting main insights
    3. Brief conclusion about significance

    Keep responses concise and well-structured.""",
    expected_output="A structured analysis with paragraph summary and bullet points",
    agent=analysis_agent
)

pretty_print("📋 Tasks created for research and analysis", "✅ Tasks Ready", "blue")


## 🧑‍🤝‍🧑 What is a Crew?

A **Crew** is a group of agents and tasks that work together. It manages who does what and how tasks are coordinated.

## 🛠️ What’s happening here?

We create a crew with:
- Two agents: one for research, one for analysis.
- Two tasks: assigned to those agents.

The crew uses delegation and iteration (`max_iter=3`) to complete the full job.


In [ ]:
# +++++ 🧑‍🤝‍🧑 Creating Crew with Agents and Tasks
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

from crewai import Crew  # Only import Crew here

# Create a crew that includes the two tasks and corresponding agents
education_crew = Crew(
    agents=[research_agent, analysis_agent],
    tasks=[research_task, analysis_task],
    verbose=False,
    max_iter=1  # Reasonable number of coordination loops
)

pretty_print("🤖 Crew assembled with Research and Analysis agents", "✅ Crew Ready", "green")


## 🚀 What is Crew Execution?

**Execution** means running the crew — agents perform their tasks in order and produce a final result.

## 🛠️ What’s happening here?

- We start execution using `.kickoff()`
- The research and analysis agents complete their tasks
- The result is printed with `pretty_print` for clear display

The current topic is shown for reference, so you always know what the crew is working on.


In [ ]:
# +++++ 🚀 Execute the Crew on Selected Topic
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Use the previously defined topic and crew
pretty_print(f"🎯 Running crew for topic: {TOPIC}", "🚀 Execution Started", "yellow")

# Run the crew and store the result
crew_result = education_crew.kickoff()

# Display the result - Convert to string first!
pretty_print(str(crew_result), "🧠 Final Output", "blue")

In [ ]:
# +++++ 🧪 Hands-On: Test with Different Topics
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Test the generalized agents with any topic you want

def analyze_topic(topic):
    """Analyze any topic using our 2-agent crew"""

    research_task_custom = Task(
        description=f"""Research the topic: {topic}

        Provide comprehensive information including:
        1. Key facts and background information
        2. Important developments and achievements
        3. Current status and recent context
        4. Significance and impact

        Use your existing knowledge to provide accurate information.""",
        expected_output=f"Research findings about: {topic}",
        agent=research_agent
    )

    analysis_task_custom = Task(
        description=f"""Analyze the research findings about: {topic}

        Create a structured summary with:
        1. One paragraph overview (3-4 sentences)
        2. 3-4 key bullet points
        3. Brief significance statement

        Keep it concise and well-structured.""",
        expected_output="Structured analysis with summary and key points",
        agent=analysis_agent
    )

    topic_crew = Crew(
        agents=[research_agent, analysis_agent],
        tasks=[research_task_custom, analysis_task_custom],
        verbose=False
    )

    result = topic_crew.kickoff()
    pretty_print(str(result), f"📋 Analysis: {topic}", "blue")
    return result

# 🎯 Test different topics - try these!
print("🎯 Try these example topics:")
print('analyze_topic("Climate Change and Global Warming")')
print('analyze_topic("Artificial Intelligence in Healthcare")')
print('analyze_topic("Space Exploration and Mars Missions")')
print('analyze_topic("Renewable Energy Technologies")')

# Example: Uncomment to test
# analyze_topic("Climate Change and Global Warming")

## 🔧 Part 2: Adding Tools to Agents

Now let's **extend our lab** by adding tools to make our agents more powerful!

In CrewAI, agents can be equipped with **tools** to perform external tasks such as:
- Searching the web 🌐
- Reading from websites 📖
- Calling APIs 📡
- Doing calculations 🧮

### 🛠️ Tool Types in CrewAI

CrewAI supports many tools via `crewai_tools` and `langchain` integration. In this part, we focus on using a **web search tool** that:
- ✅ Works without requiring an API key
- 🔓 Is fully integrated with CrewAI
- 🧠 Allows the agent to gather **current information** from the internet

---

### 🌐 WebsiteSearchTool – Free Web Search (No API Key Needed)

The `WebsiteSearchTool` is a built-in tool from `crewai_tools` that allows agents to search the web without credentials. It’s perfect for:
- Research agents looking for up-to-date content
- Summarizing recent news and trends
- Pulling data from publicly accessible websites

```python
from crewai_tools import WebsiteSearchTool

search_tool = WebsiteSearchTool()


In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import WebsiteSearchTool

# 🔍 Create a free web search agent (no API key required)
web_researcher = Agent(
    role="Web Research Agent",
    goal="Gather current online information using free CrewAI web search tool",
    backstory="You research current events with the built-in CrewAI WebsiteSearchTool (no API key needed).",
    tools=[WebsiteSearchTool()],
    verbose=True,
    allow_delegation=False
)

pretty_print("🔍 Web research agent created with WebsiteSearchTool!\n🌐 Can search websites without API keys", "✅ Web Agent Ready", "blue")

# 🎯 Task: perform and analyze search
def search_current_info(topic):
    search_task = Task(
        description=f"Use the free WebsiteSearchTool to search for recent updates on: {topic}",
        expected_output="Raw search results",
        agent=web_researcher
    )
    analysis_task = Task(
        description=f"Analyze the raw search results for: {topic}. Summarize key findings.",
        expected_output="Summary of insights",
        agent=web_researcher
    )
    crew = Crew(agents=[web_researcher], tasks=[search_task, analysis_task], verbose=False)

    pretty_print(f"🔍 Searching for current information about: {topic}", "🚀 Web Search Started", "yellow")

    result = crew.kickoff()

    pretty_print(str(result), f"🌐 Search Results: {topic}", "blue")

    return result

In [ ]:
# 🧪 Example usage:
result = search_current_info("latest AI breakthroughs 2025")

In [ ]:
# +++++ ✍️ HANDS-ON EXERCISE: Complete the Mathematical Model
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Fill in the 10 blanks to create a working mathematical analysis crew!

# Sample data: Years vs Harvard Nobel Prize winners
sample_data = "2020,2;2021,1;2022,3;2023,2;2024,1"

# Fill in the blanks below (replace ----- with your answers):

# 1. Import the Task class
from crewai import -----

# 2. Create a mathematical analysis task
math_task = Task(
    description=f"""Analyze the sample data: {sample_data}

    This data shows: Year, Number of Harvard Nobel Prize winners

    Perform linear regression analysis and provide:
    1. The mathematical relationship (slope and intercept)
    2. How well the model fits the data (R-squared)
    3. ----- about what this relationship means

    Remember: NO predictions beyond 2024!""",
    expected_output="Mathematical analysis of the Harvard Nobel Prize data",
    agent=-----  # 3. Which agent should handle this task?
)

# 4. Create a second agent for interpretation
interpretation_agent = Agent(
    role="-----",  # 5. What should this agent's role be?
    goal="Interpret mathematical results in educational context",
    backstory="""You are an expert in educational statistics who interprets mathematical
    results for educators. You focus on understanding patterns in existing data.""",
    verbose=True,
    allow_delegation=-----  # 6. Should this agent delegate? True or False?
)

# 7. Create an interpretation task
interpretation_task = Task(
    description="Interpret the mathematical regression results. Explain what the slope, intercept, and R-squared mean in the context of Harvard Nobel Prize winners. Focus on describing the relationship in the data only.",
    expected_output="Clear interpretation of the mathematical relationship",
    agent=-----  # 8. Which agent should handle this task?
)

# 9. Create a crew with both agents
math_crew = Crew(
    agents=[math_agent, -----],  # 10. Add the second agent here
    tasks=[math_task, interpretation_task],
    verbose=True
)

# Test the mathematical crew
print("🧮 Testing Mathematical Model Crew...")
print("="*50)

# Uncomment to run after filling in the blanks:
# result = math_crew.kickoff()
# pretty_print(str(result), "🧮 Mathematical Analysis", "yellow")

print("✍️ Complete the 10 blanks above, then uncomment the last 3 lines to run!")

In [ ]:
# +++++ ✅ ANSWER KEY - Mathematical Model Solution
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Complete solution for the hands-on exercise

# ANSWERS:
# 1. Task
# 2. Summary (or "Interpretation")
# 3. math_agent
# 4. Any role related to interpretation (e.g., "Data Interpreter")
# 5. "Statistical Interpreter" (or similar)
# 6. False
# 7. interpretation_task
# 8. interpretation_agent
# 9. math_crew
# 10. interpretation_agent

# Here's the complete working solution:
from crewai import Task

sample_data = "2020,2;2021,1;2022,3;2023,2;2024,1"

math_task = Task(
    description=f"""Analyze the sample data: {sample_data}

    This data shows: Year, Number of Harvard Nobel Prize winners

    Perform linear regression analysis and provide:
    1. The mathematical relationship (slope and intercept)
    2. How well the model fits the data (R-squared)
    3. Summary of what this relationship means

    Remember: NO predictions beyond 2024!""",
    expected_output="Mathematical analysis of the Harvard Nobel Prize data",
    agent=math_agent
)

interpretation_agent = Agent(
    role="Statistical Interpreter",
    goal="Interpret mathematical results in educational context",
    backstory="""You are an expert in educational statistics who interprets mathematical
    results for educators. You focus on understanding patterns in existing data.""",
    verbose=True,
    allow_delegation=False
)

interpretation_task = Task(
    description="Interpret the mathematical regression results. Explain what the slope, intercept, and R-squared mean in the context of Harvard Nobel Prize winners. Focus on describing the relationship in the data only.",
    expected_output="Clear interpretation of the mathematical relationship",
    agent=interpretation_agent
)

math_crew = Crew(
    agents=[math_agent, interpretation_agent],
    tasks=[math_task, interpretation_task],
    verbose=True
)

# Run the complete mathematical analysis
result = math_crew.kickoff()
pretty_print(str(result), "🧮 Complete Mathematical Analysis", "yellow")

## 🎓 Lab Summary

### What You've Learned:
1. **🤖 2-Agent System**: Created research and analysis agents
2. **🔧 Web Search Tool**: Added real-time information gathering
3. **🧮 Mathematical Tool**: Built custom regression analysis capability
4. **🛡️ Guardrails**: Implemented safety measures against speculation
5. **✍️ Hands-On Practice**: Completed a fill-in-the-blanks exercise

### Key Concepts:
- **Tool Integration**: How to add capabilities to agents
- **Task Coordination**: Sequential workflow between agents
- **Prompt Optimization**: Consistent, clear instructions
- **Safety Measures**: Preventing inappropriate predictions

### 🚀 Next Steps:
- Try different tools (file processing, API calls, etc.)
- Experiment with more complex mathematical models
- Create domain-specific agents for your field
- Build multi-agent workflows for your projects

**Congratulations! You've mastered CrewAI with tools and mathematical modeling! 🎉**